# Testing cartoframes

In [43]:
from carto.auth import APIKeyAuthClient
import carto
import json
creds = json.loads(open('credentials.json').read())
API_KEY = creds['api_key']
USERNAME = creds['username']
BASEURL = 'https://{username}.carto.com/api/'.format(username=USERNAME)
organization = 'team'

auth_client = APIKeyAuthClient(BASEURL, API_KEY)

## Example usage of SQLClient

In [44]:
from carto.sql import SQLClient

sql = SQLClient(auth_client)
result = sql.send('select * from all_month_3 limit 10')

### read_carto

In [46]:
import pandas as pd

def map_dtypes(pgtype):
    """
        Map PostgreSQL data types (key) to NumPy/pandas dtypes (value)
    """
    # may not be a complete list, could not find SQL API documentation
    # about data types
    dtypes = {'number': 'float64',
              'date': 'datetime64',
              'string': 'object',
              'geometry': 'object',
              'boolean': 'bool'}
    try:
        return dtypes[pgtype]
    except KeyError:
        return 'object'

def transform_schema(pgschema):
    """
        Transform schema returned via SQL API to dict for pandas
    """
    datatypes = {}
    for field in pgschema:
        if 'cartodb_id' in field:
            continue
        datatypes[field] = map_dtypes(pgschema[field]['type'])
    return datatypes

def get_geom_type(sql_auth_client, tablename):
    """
        Get the geometry type in tablename for storing in
        dataframe metadata
    """
    geomtypes = {'ST_Point': 'point',
                 'ST_MultiPoint': 'point',
                 'ST_LineString': 'line',
                 'ST_MultiLineString': 'line',
                 'ST_Polygon': 'polygon',
                 'ST_MultiPolygon': 'polygon'}

    result = sql_auth_client.send('''
        SELECT ST_GeometryType(the_geom) As geomtype
        FROM "{tablename}"
        LIMIT 1'''.format(tablename=tablename))
    try:
        return geomtypes[result['rows'][0]['geomtype']]
    except KeyError:
        print ("Cannot map ``{tablename}` because it does not have "
               "geometries").format(tablename=tablename)

# NOTE: this is compatible with v1.0.0 of carto-python client
# TODO: remove username as a param would be nice.. accessible to write to
#       metadata from carto python client?
def read_carto(cdb_client, username=None, tablename=None,
               custom_query=None, include_geom=True,
               limit=None, index='cartodb_id', debug=False):
    """Import a table from carto into a pandas dataframe, storing
       table information in pandas metadata"""
    from carto.sql import SQLClient
    import json
    sql = SQLClient(cdb_client)

    # construct query
    if tablename:
        query = 'SELECT * FROM "{tablename}"'.format(tablename=tablename)
        geomtype = get_geom_type(sql, tablename)
        # Add limit if requested
        if limit:
            # NOTE: what if limit is `all` or `none`?
            if (limit >= 0) and isinstance(limit, int):
                query += ' LIMIT {limit}'.format(limit=limit)
            else:
                raise ValueError("`limit` parameter must an integer >= 0")
    elif query:
        # NOTE: note yet implemented
        # query = custom_query
        pass
    else:
        raise NameError("`tablename` or `query` needs to be specified")

    if debug:
        print query

    # exclude geometry columns if asked
    # TODO: include_geom in cdb_client structure?

    if debug:
        print query
    # TODO: how to handle NaNs deterministically?
    resp = sql.send(query)
    schema = transform_schema(resp['fields'])
    _df = pd.DataFrame(resp['rows']).set_index(index).astype(schema)

    # TODO: add table schema to the metadata
    # NOTE: pylint complains that we're accessing a 'protected member
    #       _metadata of a client class' (appending to _metadata only works
    #       with strings, not JSON, so we're serializing here)
    _df._metadata.append(json.dumps({'carto_table': tablename,
                                     'carto_username': username,
                                     'carto_include_geom': include_geom,
                                     'carto_limit': limit,
                                     'carto_schema': str(schema),
                                     'carto_geomtype': geomtype}))
    _df.carto_last_state = _df.copy(deep=True)
    _df.carto_sql_client = sql
    return _df

pd.read_carto = read_carto

### Demo: load a dataset (without geometries) from carto

In [48]:
df = pd.read_carto(auth_client, tablename='cartoframes_test')
df.head()

Cannot map ``cartoframes_test` because it does not have geometries


,four,rand,ten,the_geom,the_geom_webmercator,three,two
cartodb_id,,,,,,,
12,8.0,95.541641,20.0,None,None,5.0,4.0
13,12.0,17.782106,30.0,None,None,7.0,6.0
14,16.0,35.715444,40.0,None,None,9.0,8.0
15,20.0,42.447783,50.0,None,None,11.0,10.0
16,24.0,16.150016,60.0,None,None,13.0,12.0


### Load a dataset with geoms from carto

In [49]:
df = pd.read_carto(auth_client, tablename='all_month_3', limit=25)
df.head()

,created_at,depth,dmin,gap,id,latitude,longitude,mag,magtype,net,nst,place,rms,the_geom,the_geom_webmercator,time,type,updated,updated_at
cartodb_id,,,,,,,,,,,,,,,,,,,
601,2015-02-17 18:36:45,8.5026,0.045,81.77000,nn00482982,39.3918,-119.8228,0.22,ml,nn,11.0,"15km S of Reno, Nevada",0.1353,0101000020E610000086C954C1A8F45DC052499D8026B2...,0101000020110F00000D079BA2FE7069C1A5D4B27DF739...,2015-02-15 01:35:33,earthquake,2015-02-15 17:21:54,2015-02-17 18:36:45
1287,2015-02-17 18:36:45,25.6000,NaN,248.39998,ak11509593,53.6875,-164.4147,2.30,ml,ak,15.0,"102km ESE of Akutan, Alaska",0.3500,0101000020E6100000D734EF38458D64C00000000000D8...,0101000020110F0000CA46ED0A667471C1535C38528620...,2015-02-12 11:25:16,earthquake,2015-02-16 20:58:06,2015-02-17 18:36:45
1383,2015-02-17 18:36:45,1.3000,NaN,NaN,ak11506296,61.2932,-152.2508,2.40,ml,ak,NaN,"84km NW of Nikiski, Alaska",0.6400,0101000020E6100000C7BAB88D060863C0F697DD9387A5...,0101000020110F0000DB3E7618D02970C1A4DC3E68DA94...,2015-02-12 03:37:53,earthquake,2015-02-16 21:00:35,2015-02-17 18:36:45
1940,2015-02-17 18:36:45,22.8000,NaN,NaN,ak11505769,60.6334,-149.4696,1.80,ml,ak,NaN,"45km WSW of Whittier, Alaska",0.5900,0101000020E6100000744694F606AF62C0A9A44E401351...,0101000020110F000075B85AF873BC6FC12C564EFEF54A...,2015-02-10 05:22:34,earthquake,2015-02-10 05:44:46,2015-02-17 18:36:45
2817,2015-02-17 18:36:45,4.6000,NaN,NaN,ak11506661,51.5992,-173.7764,1.70,ml,ak,NaN,"72km SSE of Atka, Alaska",0.3200,0101000020E6100000A913D044D8B865C0B1E1E995B2CC...,0101000020110F0000281AC2C5D37272C1F48C31664AAA...,2015-02-06 16:04:07,earthquake,2015-02-12 18:15:29,2015-02-17 18:36:45


## dataframe.update_carto code

In [51]:
def process_item(item):
    """
      Map NumPy values to PostgreSQL values
    """
    from math import isnan
    if isinstance(item, str):
        return '\'{}\''.format(item)
    elif isinstance(item, float):
        if isnan(item):
            return 'null'
        return str(item)
    return str(item)

def datatype_map(dtype):
    """
       map NumPy types to PostgreSQL types
    """
    if 'float' in dtype:
        return 'numeric'
    elif 'int' in dtype:
        return 'int'
    else:
        return 'text'

def update_carto(self, createtable=False, debug=False):
    import json
    if createtable is True:
        # TODO: build this
        # grab df schema, setup table, cartodbfy, then exit
        pass
    elif not hasattr(self, 'carto_sql_client'):
        raise Exception("Table not registered with CARTO. Set `createtable` "
                        "flag to True")
    # NOTE: remove -- handled in sql client
    # api_endpoint = 'https://{username}.carto.com/api/v2/sql?'.format(
    #     username=json.loads(self._metadata[0])['carto_username'])
    #
    # if 'carto_api_key' in json.loads(self._metadata[0]):
    #     params = {
    #         'api_key': json.loads(self._metadata[0])['carto_api_key']
    #     }
    # else:
    #     raise Exception("No API key set for this dataframe. Set with "
    #                     "update metadata method.")
    # update current state of dataframe
    # diff with the last retrieved version from carto
    # filename = 'carto_temp_{}'.format(
    #     json.loads(self._metadata[0])['carto_table'])
    # if debug: print filename
    # pd.read_csv(filename, index_col='cartodb_id')

    last_state = self.carto_last_state
    # if debug: print last_state.head()

    # create new column if needed
    # TODO: extract to function
    if len(set(self.columns) - set(last_state.columns)) > 0:
        newcols = set(self.columns) - set(last_state.columns)
        for col in newcols:
            if debug: print "Create new column {col}".format(col=col)
            alter_query = '''
                ALTER TABLE "{tablename}"
                ADD COLUMN "{colname}" {datatype}
            '''.format(tablename=json.loads(self._metadata[0])['carto_table'],
                       colname=col,
                       datatype=datatype_map(str(self.dtypes[col])))
            if debug: print alter_query
            # params['q'] = alter_query
            # add column
            # TODO: replace with `carto-python` client
            # resp = requests.get(api_endpoint + urllib.urlencode(params))
            resp = self.carto_sql_client.send(alter_query)
            # if debug: print resp.text
            # update all the values in that column
            # NOTE: fails if colval is 'inf' or some other Python or NumPy type
            for item in self[col].iteritems():
                if debug: print item
                update_query = '''
                    UPDATE {tablename}
                    SET "{colname}" = {colval}
                    WHERE "cartodb_id" = {cartodb_id};
                '''.format(tablename=json.loads(self._metadata[0])['carto_table'],
                           colname=col,
                           colval=process_item(item[1]),
                           cartodb_id=item[0])
                if debug: print update_query
                # params['q'] = update_query
                # TODO: replace with carto-python client
                # resp = requests.get(api_endpoint + urllib.urlencode(params))
                resp = self.carto_sql_client.send(update_query)
                # if debug: print resp.text
    # drop column if needed
    # TODO: extract to function
    if len(set(last_state.columns) - set(self.columns)) > 0:
        discardedcols = set(last_state.columns) - set(self.columns)
        for col in discardedcols:
            alter_query = '''
                ALTER TABLE "{tablename}"
                DROP COLUMN "{colname}"
            '''.format(tablename=json.loads(self._metadata[0])['carto_table'],
                       colname=col)
            # params['q'] = alter_query
            if debug: print alter_query
            # TODO: replace with carto-python client
            # resp = requests.get(api_endpoint + urllib.urlencode(params))
            # if debug: print resp.text
            resp = self.carto_sql_client.send(alter_query)
    # sync updated values
    # TODO: extract to column
    common_cols = list(set(self.columns) & set(last_state.columns))
    df_diff = (self[common_cols] != last_state[common_cols]).stack()
    for i in df_diff.iteritems():
        # TODO: instead of doing row by row, build up a list of queries
        #       testing to be sure the num of characters is lower than
        #       16368ish. And then run the query as a transaction
        if i[1]:
            if debug: print i
            cartodb_id = i[0][0]
            colname = i[0][1]
            upsert_query = '''
            INSERT INTO {tablename}("cartodb_id", "{colname}")
                 VALUES ({cartodb_id}, {colval})
            ON CONFLICT ("cartodb_id")
            DO UPDATE SET "{colname}" = {colval}
            WHERE EXCLUDED."cartodb_id" = {cartodb_id}
            '''.format(tablename=json.loads(self._metadata[0])['carto_table'],
                       colname=colname,
                       colval=process_item(self.loc[cartodb_id][colname]),
                       cartodb_id=cartodb_id)
            if debug: print upsert_query
            # params['q'] = upsert_query
            # TODO: replace with carto-python client
            # resp = requests.get(api_endpoint + urllib.urlencode(params))
            resp = self.carto_sql_client.send(upsert_query)
            if debug: print json.loads(resp.text)
        else:
            continue

pd.DataFrame.update_carto = update_carto

In [52]:
df = pd.read_carto(auth_client, tablename='cartoframes_test')
df.head()

Cannot map ``cartoframes_test` because it does not have geometries


,four,rand,ten,the_geom,the_geom_webmercator,three,two
cartodb_id,,,,,,,
12,8.0,95.541641,20.0,None,None,5.0,4.0
13,12.0,17.782106,30.0,None,None,7.0,6.0
14,16.0,35.715444,40.0,None,None,9.0,8.0
15,20.0,42.447783,50.0,None,None,11.0,10.0
16,24.0,16.150016,60.0,None,None,13.0,12.0


In [53]:
df['five'] = 5

In [54]:
df.head()

,four,rand,ten,the_geom,the_geom_webmercator,three,two,five
cartodb_id,,,,,,,,
12,8.0,95.541641,20.0,None,None,5.0,4.0,5
13,12.0,17.782106,30.0,None,None,7.0,6.0,5
14,16.0,35.715444,40.0,None,None,9.0,8.0,5
15,20.0,42.447783,50.0,None,None,11.0,10.0,5
16,24.0,16.150016,60.0,None,None,13.0,12.0,5


In [ ]:
df.update_carto()

## Interactive mapping experiments

### dataframe.carto_map()

In [57]:
def cartocss_by_geom(geomtype):
    if geomtype == 'point':
        markercss = '''
            #layer {
              marker-width: 7;
              marker-fill: %(filltype)s;
              marker-fill-opacity: 1;
              marker-allow-overlap: true;
              marker-line-width: 1;
              marker-line-color: #FFF;
              marker-line-opacity: 1;
            }
        '''.replace('\n', '')
        return markercss
    elif geomtype == 'line':
        linecss = '''
            #layer {
              line-width: 1.5;
              line-color: %(filltype)s;
            }
        '''.replace('\n', '')
        return linecss
    elif geomtype == 'polygon':
        polygoncss = '''
            #layer {
              polygon-fill: %(filltype)s;
              line-width: 0.5;
              line-color: #FFF;
              line-opacity: 0.5;
            }
        '''.replace('\n', '')
        return polygoncss
    return None


def get_fillstyle(params):
    """

    """

    if params['stylecol']:
        if params['datatype'] == 'float64':
            fillstyle = ('ramp([{stylecol}], cartocolor(RedOr), '
                         'quantiles())'.format(stylecol=params['stylecol']))
        else:
            fillstyle = ('ramp([{stylecol}], cartocolor(Bold), '
                         'category(10))'.format(stylecol=params['stylecol']))
    else:
        fillstyle = '#f00'

    return fillstyle


def get_mapconfig(params):
    """
        Anonymous Maps API template for carto.js
        mapconfig_params = {'username': df_meta['carto_username'],
                            'tablename': df_meta['carto_table'],
                            'geomtype': df_meta['geomtype'],
                            'stylecol': stylecol,
                            'datatype': str(self[stylecol].dtype)}
        dtypes one of
          * quantitative: float64 (float32, int32, int64)
          * categorical: bool, object
            * cartocss rule: ramp([room_type], cartocolor(Bold), category(4))
              dtypes = {'number': 'float64',
                        'date': 'datetime64',
                        'string': 'object',
                        'geometry': 'object',
                        'boolean': 'bool'}
        color palettes: https://github.com/CartoDB/CartoColor/blob/master/cartocolor.js
    """

    cartocss = cartocss_by_geom(params['geomtype']) % {'filltype': get_fillstyle(params)}

    hyperparams = dict({'cartocss': cartocss}, **params)
    # print hyperparams

    mapconfig = '''{"user_name": "%(username)s",
                    "type": "cartodb",
                    "sublayers": [{
                      "type": "http",
                      "urlTemplate": "http://{s}.basemaps.cartocdn.com/dark_nolabels/{z}/{x}/{y}.png"
                      }, {
                      "sql": "select * from %(tablename)s",
                      "cartocss": "%(cartocss)s"
                      }],
                      "subdomains": [ "a", "b", "c" ]
                      }''' % hyperparams

    return mapconfig

def carto_map(self, interactive=True, stylecol=None):
    """
        Produce and return CARTO maps or iframe embeds
    """
    import urllib
    import json
    import IPython
    return_iframe = False
    # create static map
    if interactive is False:
        # TODO: use carto-python client to create static map (not yet
        #       implemented)
        raise NotImplementedError("This feature is not yet implemented")

    df_meta = json.loads(self._metadata[-1])
    mapconfig_params = {'username': df_meta['carto_username'],
                        'tablename': df_meta['carto_table'],
                        'geomtype': df_meta['carto_geomtype'],
                        'stylecol': stylecol,
                        'datatype': (str(self[stylecol].dtype)
                                     if stylecol in self.columns
                                     else None)}

    mapconfig = get_mapconfig(mapconfig_params)
    # TODO: include in uriencode in mapconfig?
    mapconfig_params['q'] = urllib.quote(mapconfig)

    # print params
    url = '?'.join(['/files/cartoframes.html',
                    urllib.urlencode(mapconfig_params)])
    iframe = '<iframe src="{url}" width=700 height=350></iframe>'.format(url=url)
    if return_iframe is True:
        return iframe
    return IPython.display.HTML(iframe)

pd.DataFrame.carto_map = carto_map


In [29]:
# brooklyn_poverty poverty_per_pop
# dsny_monthly_tonnage_data
# us_counties pop_sqkm
# ny_boroughs boroname

## The following outputs an interactive map

In [59]:
df = pd.read_carto(auth_client, tablename='us_counties', username='eschbacher')
df.carto_map(interactive=True, stylecol='pop_sqkm')